In [4]:
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anhto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df = pd.read_csv("spotify_millsongdata.csv")

In [6]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [7]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [8]:
df.shape

(57650, 4)

In [9]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [10]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [11]:
df.head(10)

,artist,song,text
0,Eurythmics,Cold,Come to me \r\nDo and be done with me \r\n(C...
1,Roy Orbison,City Life,"Coffer cafe's dizzy with city lights, concrete..."
2,'n Sync,Circle Of Life,From the day we arrive on the planet \r\nAnd ...
3,John Prine,Wedding Bells,"I got the invitation that you sent me, \r\nYo..."
4,Jimmy Buffett,Everybody's Got A Cousin In Miami,It was ninety miles to freedom but they took t...
5,Meat Loaf,Not A Dry Eye In The House,Not a dry eye in the house. \r\nAfter loves c...
6,Loretta Lynn,Another Man Loved Me Last Night,He doesn't know that I can't look him in the e...
7,John Denver,Away In The Manger,Away in a manger no crib for his bed. \r\nThe...
8,Rolling Stones,Emotional Rescue,"Is there nothing I can say, nothing I can do t..."
9,Z-Ro,Look At Me,"[Z-Ro: talking] \r\n3rd Coast, Screwston Texa..."


In [12]:
df['text'][0]

"Come to me  \r\nDo and be done with me  \r\n(Cold cold cold)  \r\nDon't I exist for you  \r\nDon't I still live for you  \r\n(Cold cold cold)  \r\nEverything I possess  \r\nGiven with tenderness  \r\nWrapped in a ribbon of glass  \r\nTime it may take us but God only knows  \r\nHow I've paid for those things in the past  \r\nDying is easy it's living that scares me to death  \r\nI could be so content hearing the sound of your breath  \r\nCold is the colour of crystal the snowlight  \r\nThat falls from the heavenly skies  \r\nCatch me and let me dive under  \r\nFor I want to swim in the pools of your eyes  \r\nI want to be with you baby  \r\nSlip me inside of your heart  \r\nDon't I belong to you baby  \r\nDon't you know that nothing can tear us apart  \r\nCome on now come on now come on now  \r\nTelling you that  \r\nI loved you right from the start...  \r\nBut the more I want you the less I get  \r\nAin't that just the way things are...  \r\n  \r\nWinter has frozen us  \r\nLet love ta

In [13]:
# df = df.sample(5000)

In [14]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [15]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [16]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [17]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [20]:
similarity[0]

array([1.        , 0.03125047, 0.01058506, ..., 0.07017625, 0.04399546,
       0.05226608])

In [21]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [22]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [25]:
recommendation('Crying Over You')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [26]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))